In [ ]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.svm import LinearSVC, OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import eli5
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
np.random.seed(500)
nltk.download('punkt')

In [ ]:
df = pd.read_pickle('../data/v2.0-processed/df_premises.pickle')
df_test = pd.read_pickle('../data/v2.0-processed/df_premises_test.pickle')
df.head()
df_test.head()

In [ ]:
# Lowercase
df['premise'] = df['premise'].str.lower()
df['logos'] = df['logos'].map({True: 'Logos', False: 'Not Logos'})
df['pathos'] = df['pathos'].map({True: 'Pathos', False: 'Not Pathos'})
df['ethos'] = df['ethos'].map({True: 'Ethos', False: 'Not Ethos'})
df['premise'] = [word_tokenize(entry) for entry in df['premise']]
df['text'] = [' '.join(entry) for entry in df['premise']]

df_test['premise'] = df_test['premise'].str.lower()
df_test['logos'] = df_test['logos'].map({True: 'Logos', False: 'Not Logos'})
df_test['pathos'] = df_test['pathos'].map({True: 'Pathos', False: 'Not Pathos'})
df_test['ethos'] = df_test['ethos'].map({True: 'Ethos', False: 'Not Ethos'})
df_test['premise'] = [word_tokenize(entry) for entry in df_test['premise']]
df_test['text'] = [' '.join(entry) for entry in df_test['premise']]

In [ ]:
df['logos'].value_counts()
df_test['logos'].value_counts()

In [ ]:
# train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df['type'], shuffle=True)

In [ ]:
# sem_type = 'pathos'
# train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df[sem_type])
# print(train[sem_type].value_counts())
# print(test[sem_type].value_counts())
# Train_X, Test_X, Train_Y, Test_Y = train['text'], test['text'], train[sem_type], test[sem_type]
# Encoder = LabelEncoder()
# Train_Y = Encoder.fit_transform(Train_Y)
# Test_Y = Encoder.fit_transform(Test_Y)
# print(np.count_nonzero(Train_Y == 1), np.count_nonzero(Train_Y == 0))
# print(np.count_nonzero(Test_Y == 1), np.count_nonzero(Test_Y == 0))
# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(df['text'])
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)
# # Classifier - Algorithm - SVM
# # fit the training dataset on the classifier
# SVM = svm.LinearSVC(C=1.0)
# SVM.fit(Train_X_Tfidf, Train_Y) # predict the labels on validation dataset
# predictions_SVM = SVM.predict(Test_X_Tfidf) # Use accuracy_score function to get the accuracy
# # y = Encoder.inverse_transform(Test_Y)
# # pred = Encoder.inverse_transform(predictions_SVM)
# # print(f"SVM Accuracy Score for {sem_type} -> \n", classification_report(pred, y))
# print(f"SVM Accuracy Score for {sem_type} -> \n", classification_report(Test_Y, predictions_SVM))

In [ ]:
train, dev = train_test_split(df, test_size=0.2, random_state=0, stratify=df['type'], shuffle=True)
test = df_test


print(test['pathos'].value_counts())
print(test['logos'].value_counts())
print(test['ethos'].value_counts())

def train_svm(sem_type):
    Train_X, Test_X, Dev_X, Train_Y, Test_Y, Dev_Y,  = train['text'], test['text'], dev['text'], train[sem_type], test[sem_type], dev[sem_type]

    # Construct pipeline
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('svc', LinearSVC()),
    ])

    pipe.fit(Train_X, Train_Y)

    predictions_SVM_dev = pipe.predict(Dev_X)
    predictions_SVM_test = pipe.predict(Test_X) # Use accuracy_score function to get the accuracy

    print(f"{sem_type} (dev set) -> \n", classification_report(Dev_Y, predictions_SVM_dev))

    print(f"{sem_type} (test set) -> \n", classification_report(Test_Y, predictions_SVM_test))

    eli5.show_weights(pipe)

In [ ]:
for sem_type in ['logos', 'ethos', 'pathos']:
    train_svm(sem_type=sem_type)